In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler,MinMaxScaler
X_ppg,X_qual,X_acc,y_rr, y_respiration,y_inspiration,y_expiration,groups,X_respiration,y_activity,y_label = pickle.load(open('../../affect-recognition/data/tabular_data_8.p','rb'))

In [2]:
from sklearn.preprocessing import OneHotEncoder
index = np.where((y_activity!=0)&(y_activity!=3)&(y_activity!=6)&(y_activity!=8))[0]
X_acc = X_acc[index]
y_activity = y_activity[index]
groups = groups[index]
y_activity = OneHotEncoder().fit_transform(y_activity.reshape(-1,1)).todense()
# X_fft = np.fft.fft(X_acc,axis=1)
# X_fft_real = X_fft.real
# X_fft_imag = X_fft.imag
# X_fft = np.concatenate([X_fft_real,X_fft_imag],axis=2)
# X_fft = X_fft_real

/home/azim/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
from keras.layers import Input, LSTM, RepeatVector,Bidirectional,Multiply,multiply,Permute
from keras.layers import TimeDistributed,Dense,Flatten,Reshape,Lambda,Activation,GRU,Conv1D,MaxPool1D,Dropout,BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
from keras import metrics,losses
import tensorflow as tf
# import tensorflow_probability as tf
from sklearn.model_selection import GroupKFold

def get_base_CNN(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='kullback_leibler_divergence',output_dim=5,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    fe = Conv1D(200,10, padding='same',activation='linear')(inputs)
#     fe = MaxPool1D(2)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.1)(fe)
    fe = Conv1D(100,10, padding='same',activation='tanh')(fe)
#     fe = MaxPool1D(2)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.1)(fe)
    fe = Conv1D(200,10, padding='same',activation='tanh')(fe)
#     fe = MaxPool1D(4)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.1)(fe)
    fe = Bidirectional(LSTM(60,return_sequences=True,activation='tanh'))(fe)
    fe = Bidirectional(LSTM(60,return_sequences=False,activation='tanh'))(fe)
#     fe = TimeDistributed(Dense(10))(fe)
#     flattened = Flatten()(fe)
    output = Dense(30,activation='tanh')(fe)
    output = Dense(output_dim,activation='sigmoid')(output)
    output = Dense(output_dim,activation='softmax')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder


gkf = GroupKFold(n_splits=len(np.unique(groups)))
indexes = list(gkf.split(X_acc,groups=groups))
results = []
for i,ii in enumerate(indexes):
    train,test = ii
    X_acc_train,X_acc_test = X_acc[train],X_acc[test]
    y_activity_train,y_activity_test = y_activity[train],y_activity[test]
    y_activity_train.shape
    sequence_autoencoder = get_base_CNN()
    sequence_autoencoder.summary()
    filepath = '../../affect-recognition/data/models/activity/'+str(groups[test][0])+'v4.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True,save_weights_only=False, mode='max')
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1,patience=30)
    callbacks_list = [es,checkpoint]
    history = sequence_autoencoder.fit(X_acc_train, y_activity_train,
                    epochs=400,
                    batch_size=100,
                    shuffle=True,
                    validation_split=.3,callbacks=callbacks_list)
    %matplotlib notebook
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from keras.models import load_model
    sequence_autoencoder.load_weights(filepath)
    y_activity_pred = sequence_autoencoder.predict(X_acc_test)
    results.append([i,y_activity_test,y_activity_pred])
    print(confusion_matrix(np.int64(np.argmax(y_activity_test,axis=1)),np.int64(np.argmax(y_activity_pred,axis=1))))

Model: "model_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        (None, 256, 3)            0         
_________________________________________________________________
conv1d_162 (Conv1D)          (None, 256, 200)          6200      
_________________________________________________________________
batch_normalization_162 (Bat (None, 256, 200)          800       
_________________________________________________________________
dropout_162 (Dropout)        (None, 256, 200)          0         
_________________________________________________________________
conv1d_163 (Conv1D)          (None, 256, 100)          200100    
_________________________________________________________________
batch_normalization_163 (Bat (None, 256, 100)          400       
_________________________________________________________________
dropout_163 (Dropout)        (None, 256, 100)          0  

14732/14732 [==============================] - 115s 8ms/step - loss: 0.4111 - accuracy: 0.8029 - val_loss: 0.8509 - val_accuracy: 0.7061

Epoch 00026: val_accuracy did not improve from 0.77463
Epoch 27/400
14732/14732 [==============================] - 115s 8ms/step - loss: 0.4765 - accuracy: 0.7839 - val_loss: 0.8177 - val_accuracy: 0.7209

Epoch 00027: val_accuracy did not improve from 0.77463
Epoch 28/400
14732/14732 [==============================] - 115s 8ms/step - loss: 0.4208 - accuracy: 0.8010 - val_loss: 0.7929 - val_accuracy: 0.7162

Epoch 00028: val_accuracy did not improve from 0.77463
Epoch 29/400
14732/14732 [==============================] - 115s 8ms/step - loss: 0.4426 - accuracy: 0.7885 - val_loss: 0.8129 - val_accuracy: 0.6863

Epoch 00029: val_accuracy did not improve from 0.77463
Epoch 30/400
14732/14732 [==============================] - 115s 8ms/step - loss: 0.4110 - accuracy: 0.8004 - val_loss: 0.8691 - val_accuracy: 0.6703

Epoch 00030: val_accuracy did not impr

In [ ]:
# 0.h5 4 class model
#0v1.h5 accurate 4 class model
#0v2.h5 5 class model
#0v3.h5 5 class more accurate model
#0.v4 5 class LSTM model

pickle.dump(results,open('../../affect-recognition/data/activity_results4.p','wb'))

In [ ]:
# conf = np.zeros((5,5))
y_true_all = []
y_pred_all = []
scores = []
all_data = []
for a in results:
    i,y_true,y_pred = a
    y_true = np.array(np.int64(np.argmax(y_true,axis=1))).reshape(-1)
    y_pred = np.array(np.int64(np.argmax(y_pred,axis=1))).reshape(-1)
    y_true_all.extend(list(y_true))
    y_pred_all.extend(list(y_pred))
    scores.append(np.array([f1_score(y_true,y_pred,average='macro'),
                            balanced_accuracy_score(y_true,y_pred),
                            accuracy_score(y_true,y_pred)]))
    all_data.append([scores[-1][0],'Macro F1'])
    all_data.append([scores[-1][1],'Balanced Accuracy'])
    all_data.append([scores[-1][2],'Accuracy'])
    
    
#     if confusion_matrix(y_true,y_pred).shape[0]!=5:
#         continue
#     conf+=confusion_matrix(y_true,y_pred)

In [ ]:
import pandas as pd
df = pd.DataFrame(all_data,columns=['Value',''])

In [ ]:
scores = np.array(scores)
plt.rcParams.update({'font.size':25})
plt.figure(figsize=(10,8))
sns.boxplot(x='',y='Value',data=df)
plt.xticks(rotation=5)
plt.savefig('../../affect-recognition/data/pics/boxplot.pdf',dps=1e6)
plt.show()
np.median(scores,axis=0),scores

In [ ]:
conf = confusion_matrix(y_true_all,y_pred_all)

In [ ]:
conf = conf/np.sum(conf,axis=1)

In [ ]:
# np.array(y_true).reshape(-1)
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
f1_score(y_true_all,y_pred_all,average='macro'),balanced_accuracy_score(y_true_all,y_pred_all)

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,7))
# tmp = np.int64(confusion_matrix(y_test_all,y_pred_all))//98
sns.heatmap(conf,annot=True,fmt='.2f',annot_kws={"fontsize":22},cmap=None,vmin=0,vmax=1,cbar=False)
plt.xticks([.5,1.5,2.5,3.5,4.5],['Sitting','Stairs','Cycling','Driving','Walking'],fontsize=22)
plt.yticks([.5,1.5,2.5,3.5,4.5],['Sitting','Stairs','Cycling','Driving','Walking'],rotation=0,fontsize=22)
plt.savefig('../../affect-recognition/data/pics/confusion_matrix.pdf',dps=1e6)
# plt.yticks([0.5,1.5],['LAB','FIELD'])
# plt.title('Average Confusion Matrix')
plt.show()

In [ ]:
def get_base_GRU(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='binary_crossentropy',output_dim=9,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    encoded = Bidirectional(GRU(first_dim,return_sequences=True,activation='tanh',go_backwards=True))(inputs)
    dense_layer = TimeDistributed(Dense(1,activation='tanh'))(encoded)
    flattened = Flatten()(dense_layer)
    output = Dense(30,activation='relu')(flattened)
    output = Dense(output_dim,activation='sigmoid')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder


def get_base_CNN(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='kullback_leibler_divergence',output_dim=4,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    fe = Conv1D(100,4, padding='same',activation='tanh')(inputs)
    fe = MaxPool1D(2)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(100,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(4)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(50,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(8)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    flattened = Flatten()(fe)
    output = Dense(30,activation='relu')(flattened)
    output = Dense(output_dim,activation='sigmoid')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder

# timesteps_ppg = 256
# input_dim_ppg = 3
# sequence_autoencoder = get_base_autoencoder(timesteps_ppg=timesteps_ppg,input_dim_ppg=input_dim_ppg,
#                                     first_dim=60,optimizer='adam',loss='mse')

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(X_acc_test[20])
# plt.plot(X_ppg_pred[330],'r')
plt.show()

In [ ]:
y_rr_pred = sequence_autoencoder.predict(X_ppg_test)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_rr_pred)
plt.plot(y_rr_test,'r')

In [ ]:
differences = np.abs(y_rr_pred-y_rr_test)

In [ ]:
relative_powers = [np.mean(a[:,-1]) for a in X_qual_test]

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(relative_powers,differences)
plt.show()

In [ ]:
np.mean(differences)